In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pdb
from scipy.optimize import minimize
from scipy.stats import multivariate_normal
from tqdm.notebook import tqdm
import random
import pandas as pd
from scipy.io import savemat
import scipy
import matplotlib.cm as cm
import os
import glob
from utils import *

In [2]:
def dataCleaning(csv_test,csv_data,total_number_possible_responses):

    n_tones = 3
    n_trials = csv_data.shape[0]-47

    """
    Get tones and values of keys pressed
    """
    test_columns = list(csv_test.columns)
    test_tones_name = test_columns.index('Name')
    test_tones_col_idx = test_columns.index('Tones')
    test_tones_cat_col_idx = test_columns.index('Tonekind')

    df_names = (csv_test.iloc[:,test_tones_name]).values
    df_tones = (csv_test.iloc[:,test_tones_col_idx]).values
    df_tone_cat = (csv_test.iloc[:,test_tones_cat_col_idx]).values

    tones_array_orig = np.zeros((n_trials,n_tones))
    tones_array_idxs_keep = []

    tones_cat_array_orig = np.zeros((n_trials,n_tones))
    tones_cat_array_idxs_keep = []

    for i_wav in range(n_trials):
        if isinstance(csv_data['Name'][i_wav+46],str):
            tones_array_orig[i_wav,:] = np.array(df_tones[np.where(csv_data['Name'][i_wav+46]\
                                                              ==df_names)[0]][0][1:-1].split(',')).astype(float)  
            tones_array_idxs_keep += [i_wav]

            tones_cat_array_orig[i_wav,:] = np.array(df_tone_cat[np.where(csv_data['Name'][i_wav+46]\
                                                              ==df_names)[0]][0][1:-1].split(',')).astype(float)  
            tones_cat_array_idxs_keep += [i_wav]


    df_tones = np.copy(tones_array_orig[tones_array_idxs_keep,:])
    df_tone_cat = np.copy(tones_cat_array_orig[tones_cat_array_idxs_keep,:])
    df_corrans = np.copy(csv_data['corrAns'][46:csv_data.shape[0]])[tones_array_idxs_keep]
    df_keys = np.copy(csv_data['test_resp.keys'][46:csv_data.shape[0]])[tones_array_idxs_keep]
    
    """
    Find no response cases in the expt
    """
    no_response = np.intersect1d(np.where(df_keys!='h')[0],np.where(df_keys!='l')[0])
    print("Did not respond to: ",no_response)

    """
    Convert keys ['l','h'] to [0,1] and plot p(H|T)
    """
    corrans_num_orig = np.zeros_like(df_corrans)
    corrans_num_orig[df_corrans == 'h'] = 1

    keys_num_orig = np.zeros_like(df_keys)
    keys_num_orig[df_keys == 'h'] = 1

    corrans_num = corrans_num_orig[:total_number_possible_responses]
    keys_num = keys_num_orig[:total_number_possible_responses]
    tones_array = df_tones[:total_number_possible_responses,:]
    tone_cat_array = df_tone_cat[:total_number_possible_responses,:]

    trial_tones = np.repeat(tones_array,1,axis = 0)
    trial_tone_cat = np.repeat(tone_cat_array,1,axis = 0)
    trial_behaviour = np.reshape(keys_num,np.prod(keys_num.shape)) 
    correct_answer = np.reshape(corrans_num,np.prod(corrans_num.shape)) 
                
    trial_tones[no_response,:] = np.array([np.nan,np.nan,np.nan])
    trial_tone_cat[no_response,:] = np.array([np.nan,np.nan,np.nan])
    trial_behaviour[no_response] = np.nan
    correct_answer[no_response] = np.nan
    
    return trial_tones, trial_behaviour, trial_tone_cat, correct_answer


In [3]:
freqSeq = np.arange(90,3000,1) #array of possible true tones
exptFreqSeqArray = np.arange(np.log10(freqSeq[0]), np.log10(freqSeq[-1]), np.log10(1003/1000)*40)
logFreqBins = exptFreqSeqArray[np.arange(1,len(exptFreqSeqArray),2)]#(exptFreqSeqArray[:-1]+exptFreqSeqArray[1:])/2
expt_freq_seq_mid = np.median(exptFreqSeqArray)
low_dist = [expt_freq_seq_mid - 0.15,0.1]
high_dist = [expt_freq_seq_mid + 0.15,0.1]

def trialCategories(trial_tones, trial_behaviour, corrans, trial_tones_kind):
    """
    dividing trials into different combinations : 
    1. all trials with tones from low gaussian distribution that are then followed by trials with 
    tones from high gaussian distribution
    2. all trials with tones from high gaussian distribution that are then followed by trials with 
    tones from low gaussian distribution
    3. all trials with tones from low gaussian distribution that are then followed by trials with 
    1 distractor tone from high gauss and higher extreme
    4. all trials with tones from high gaussian distribution that are then followed by trials with 
    1 distractor tone from low gauss and lower extreme
    """
    
    trialsGaussLowGaussHigh = np.zeros((len(trial_behaviour),5))
    trialsGaussLowGaussLow = np.zeros((len(trial_behaviour),5))
    trialsGaussHighGaussLow = np.zeros((len(trial_behaviour),5))
    trialsGaussHighGaussHigh = np.zeros((len(trial_behaviour),5))
    trialsGaussLowGaussDistractorLow = np.zeros((len(trial_behaviour),5))
    trialsGaussHighGaussDistractorHigh = np.zeros((len(trial_behaviour),5))
    trialsAllLowMinOneSignalLow = np.zeros((len(trial_behaviour),5))
    trialsAllHighMinOneSignalHigh = np.zeros((len(trial_behaviour),5))
    trialsAllLowMinOneSignalHigh = np.zeros((len(trial_behaviour),5))
    trialsAllHighMinOneSignalLow = np.zeros((len(trial_behaviour),5))
    trialsGaussLowDistractorLow = np.zeros((len(trial_behaviour),5))
    trialsGaussHighDistractorHigh = np.zeros((len(trial_behaviour),5))
    trialsGaussLowDistractorHigh = np.zeros((len(trial_behaviour),5))
    trialsGaussHighDistractorLow = np.zeros((len(trial_behaviour),5))
    twoLowTrialsGaussDistractorLow = np.zeros((len(trial_behaviour),5))
        
    n_tones = len(trial_tones[0])
    print(high_dist[0]+2*high_dist[1], 
          low_dist[0]-2*low_dist[1], 
          expt_freq_seq_mid)
    
    iTrialsGaussLowGaussHigh = 0; iTrialsGaussHighGaussLow = 0; 
    iTrialsGaussLowGaussLow = 0; iTrialsGaussHighGaussHigh = 0; 
    iTrialsGaussLowDistractorLow = 0; iTrialsGaussHighDistractorHigh = 0;
    iTrialsGaussLowDistractorHigh = 0; iTrialsGaussHighDistractorLow = 0;
    iTrialsGaussLowGaussDistractorLow = 0; iTrialsGaussHighGaussDistractorHigh = 0
    iTrialsAllLowMinOneSignalLow = 0; iTrialsAllHighMinOneSignalHigh = 0;
    iTrialsAllLowMinOneSignalHigh = 0; iTrialsAllHighMinOneSignalLow = 0;
    iTwoLowTrialsGaussDistractorLow = 0;
    
    for i_trial in range(1,len(trial_tones)):
        if ((corrans[i_trial-1] == 1) and (corrans[i_trial]==0)):
            if (np.all(trial_tones_kind[i_trial-1,:]) and np.all(trial_tones_kind[i_trial,:])):
                trialsGaussHighGaussLow[iTrialsGaussHighGaussLow,:3] = trial_tones[i_trial,:]
                trialsGaussHighGaussLow[iTrialsGaussHighGaussLow,3] = trial_behaviour[i_trial]
                trialsGaussHighGaussLow[iTrialsGaussHighGaussLow,4] = corrans[i_trial]
                iTrialsGaussHighGaussLow += 1 
        if ((corrans[i_trial-1] == 1) and (corrans[i_trial]==1)):
            if (np.all(trial_tones_kind[i_trial-1,:]) and np.all(trial_tones_kind[i_trial,:])):
                trialsGaussHighGaussHigh[iTrialsGaussHighGaussHigh,:3] = trial_tones[i_trial,:]
                trialsGaussHighGaussHigh[iTrialsGaussHighGaussHigh,3] = trial_behaviour[i_trial]
                trialsGaussHighGaussHigh[iTrialsGaussHighGaussHigh,4] = corrans[i_trial]
                iTrialsGaussHighGaussHigh += 1 
        if ((corrans[i_trial-1] == 0) and (corrans[i_trial]==1)):
            if (np.all(trial_tones_kind[i_trial-1,:]) and np.all(trial_tones_kind[i_trial,:])):
                trialsGaussLowGaussHigh[iTrialsGaussLowGaussHigh,:3] = trial_tones[i_trial,:]
                trialsGaussLowGaussHigh[iTrialsGaussLowGaussHigh,3] = trial_behaviour[i_trial]
                trialsGaussLowGaussHigh[iTrialsGaussLowGaussHigh,4] = corrans[i_trial]
                iTrialsGaussLowGaussHigh += 1 
        if ((corrans[i_trial-1] == 0) and (corrans[i_trial]==0)):
            if (np.all(trial_tones_kind[i_trial-1,:]) and np.all(trial_tones_kind[i_trial,:])):
                trialsGaussLowGaussLow[iTrialsGaussLowGaussLow,:3] = trial_tones[i_trial,:]
                trialsGaussLowGaussLow[iTrialsGaussLowGaussLow,3] = trial_behaviour[i_trial]
                trialsGaussLowGaussLow[iTrialsGaussLowGaussLow,4] = corrans[i_trial]
                iTrialsGaussLowGaussLow += 1 
        if ((corrans[i_trial-1] == 1) and (corrans[i_trial]==1)):
            if (np.all(trial_tones_kind[i_trial-1,:]) and np.sum(trial_tones_kind[i_trial]==0)<3):
                trialsGaussHighGaussDistractorHigh[iTrialsGaussHighGaussDistractorHigh,:3] = trial_tones[i_trial,:]
                trialsGaussHighGaussDistractorHigh[iTrialsGaussHighGaussDistractorHigh,3] = trial_behaviour[i_trial]
                trialsGaussHighGaussDistractorHigh[iTrialsGaussHighGaussDistractorHigh,4] = corrans[i_trial]
                iTrialsGaussHighGaussDistractorHigh += 1 
        if ((corrans[i_trial-1] == 0) and (corrans[i_trial]==0)):
            if (np.all(trial_tones_kind[i_trial-1,:]) and np.sum(trial_tones_kind[i_trial]==0)<3):
                trialsGaussLowGaussDistractorLow[iTrialsGaussLowGaussDistractorLow,:3] = trial_tones[i_trial,:]
                trialsGaussLowGaussDistractorLow[iTrialsGaussLowGaussDistractorLow,3] = trial_behaviour[i_trial]
                trialsGaussLowGaussDistractorLow[iTrialsGaussLowGaussDistractorLow,4] = corrans[i_trial]
                iTrialsGaussLowGaussDistractorLow += 1 
        if ((corrans[i_trial-1] == 1) and (corrans[i_trial]==1)):
            if np.sum(trial_tones_kind[i_trial]==0)<3:
                trialsAllHighMinOneSignalHigh[iTrialsAllHighMinOneSignalHigh,:3] = trial_tones[i_trial,:]
                trialsAllHighMinOneSignalHigh[iTrialsAllHighMinOneSignalHigh,3] = trial_behaviour[i_trial]
                trialsAllHighMinOneSignalHigh[iTrialsAllHighMinOneSignalHigh,4] = corrans[i_trial]
                iTrialsAllHighMinOneSignalHigh += 1 
        if ((corrans[i_trial-1] == 0) and (corrans[i_trial]==0)):
            if np.sum(trial_tones_kind[i_trial]==0)<3:
                trialsAllLowMinOneSignalLow[iTrialsAllLowMinOneSignalLow,:3] = trial_tones[i_trial,:]
                trialsAllLowMinOneSignalLow[iTrialsAllLowMinOneSignalLow,3] = trial_behaviour[i_trial]
                trialsAllLowMinOneSignalLow[iTrialsAllLowMinOneSignalLow,4] = corrans[i_trial]
                iTrialsAllLowMinOneSignalLow += 1 
        if ((corrans[i_trial-1] == 1) and (corrans[i_trial]==0)):
            if np.sum(trial_tones_kind[i_trial]==0)<3:
                trialsAllHighMinOneSignalLow[iTrialsAllHighMinOneSignalLow,:3] = trial_tones[i_trial,:]
                trialsAllHighMinOneSignalLow[iTrialsAllHighMinOneSignalLow,3] = trial_behaviour[i_trial]
                trialsAllHighMinOneSignalLow[iTrialsAllHighMinOneSignalLow,4] = corrans[i_trial]
                iTrialsAllHighMinOneSignalLow += 1 
        if ((corrans[i_trial-1] == 0) and (corrans[i_trial]==1)):
            if np.sum(trial_tones_kind[i_trial]==0)<3:
                trialsAllLowMinOneSignalHigh[iTrialsAllLowMinOneSignalHigh,:3] = trial_tones[i_trial,:]
                trialsAllLowMinOneSignalHigh[iTrialsAllLowMinOneSignalHigh,3] = trial_behaviour[i_trial]
                trialsAllLowMinOneSignalHigh[iTrialsAllLowMinOneSignalHigh,4] = corrans[i_trial]
                iTrialsAllLowMinOneSignalHigh += 1 
        if ((corrans[i_trial-1] == 1) and (corrans[i_trial]==1)):
            if (np.sum(trial_tones_kind[i_trial-1]==0)==0 and 
            (np.sum(trial_tones_kind[i_trial]==0)==1 or np.sum(trial_tones_kind[i_trial]==0)==2)):
                trialsGaussHighDistractorHigh[iTrialsGaussHighDistractorHigh,:3] = trial_tones[i_trial,:]
                trialsGaussHighDistractorHigh[iTrialsGaussHighDistractorHigh,3] = trial_behaviour[i_trial]
                trialsGaussHighDistractorHigh[iTrialsGaussHighDistractorHigh,4] = corrans[i_trial]
                iTrialsGaussHighDistractorHigh += 1
        if ((corrans[i_trial-1] == 0) and (corrans[i_trial]==0)):
            if (np.sum(trial_tones_kind[i_trial-1]==0)==0 and 
            (np.sum(trial_tones_kind[i_trial]==0)==1 or np.sum(trial_tones_kind[i_trial]==0)==2)):
                trialsGaussLowDistractorLow[iTrialsGaussLowDistractorLow,:3] = trial_tones[i_trial,:]
                trialsGaussLowDistractorLow[iTrialsGaussLowDistractorLow,3] = trial_behaviour[i_trial]
                trialsGaussLowDistractorLow[iTrialsGaussLowDistractorLow,4] = corrans[i_trial]
                iTrialsGaussLowDistractorLow += 1 
        if ((corrans[i_trial-1] == 0) and (corrans[i_trial]==1)):
            if (np.sum(trial_tones_kind[i_trial-1]==0)==0 and 
            (np.sum(trial_tones_kind[i_trial]==0)==1 or np.sum(trial_tones_kind[i_trial]==0)==2)):
                trialsGaussLowDistractorHigh[iTrialsGaussLowDistractorHigh,:3] = trial_tones[i_trial,:]
                trialsGaussLowDistractorHigh[iTrialsGaussLowDistractorHigh,3] = trial_behaviour[i_trial]
                trialsGaussLowDistractorHigh[iTrialsGaussLowDistractorHigh,4] = corrans[i_trial]
                iTrialsGaussLowDistractorHigh += 1
        if ((corrans[i_trial-1] == 1) and (corrans[i_trial]==0)):
            if (np.sum(trial_tones_kind[i_trial-1]==0)==0 and 
            (np.sum(trial_tones_kind[i_trial]==0)==1 or np.sum(trial_tones_kind[i_trial]==0)==2)):
                trialsGaussHighDistractorLow[iTrialsGaussHighDistractorLow,:3] = trial_tones[i_trial,:]
                trialsGaussHighDistractorLow[iTrialsGaussHighDistractorLow,3] = trial_behaviour[i_trial]
                trialsGaussHighDistractorLow[iTrialsGaussHighDistractorLow,4] = corrans[i_trial]
                iTrialsGaussHighDistractorLow += 1

    for i_trial in range(2,len(trial_tones)):    
        if ((corrans[i_trial-2] == 0) and (corrans[i_trial-1] == 0) and (corrans[i_trial]==0)):
            if np.sum(trial_tones_kind[i_trial]==0)<2:
            #if (np.all(trial_tones_kind[i_trial,:]) and np.sum(trial_tones_kind[i_trial+1]==0)<2):
                twoLowTrialsGaussDistractorLow[iTwoLowTrialsGaussDistractorLow,:3] = trial_tones[i_trial,:]
                twoLowTrialsGaussDistractorLow[iTwoLowTrialsGaussDistractorLow,3] = trial_behaviour[i_trial]
                twoLowTrialsGaussDistractorLow[iTwoLowTrialsGaussDistractorLow,4] = corrans[i_trial]
                iTwoLowTrialsGaussDistractorLow += 1 
                
    return(trialsGaussHighGaussLow[:iTrialsGaussHighGaussLow,:],
           trialsGaussHighGaussHigh[:iTrialsGaussHighGaussHigh,:],
           trialsGaussLowGaussHigh[:iTrialsGaussLowGaussHigh,:],
           trialsGaussLowGaussLow[:iTrialsGaussLowGaussLow,:],
           trialsGaussHighGaussDistractorHigh[:iTrialsGaussHighGaussDistractorHigh,:],
           trialsGaussLowGaussDistractorLow[:iTrialsGaussLowGaussDistractorLow,:],
           trialsAllHighMinOneSignalHigh[:iTrialsAllHighMinOneSignalHigh,:],
           trialsAllLowMinOneSignalLow[:iTrialsAllLowMinOneSignalLow,:],
           trialsAllHighMinOneSignalLow[:iTrialsAllHighMinOneSignalLow,:],
           trialsAllLowMinOneSignalHigh[:iTrialsAllLowMinOneSignalHigh,:],
           trialsGaussHighDistractorHigh[:iTrialsGaussHighDistractorHigh,:],
           trialsGaussLowDistractorLow[:iTrialsGaussLowDistractorLow,:],
           trialsGaussHighDistractorLow[:iTrialsGaussHighDistractorLow,:],
           trialsGaussLowDistractorHigh[:iTrialsGaussLowDistractorHigh,:],
           twoLowTrialsGaussDistractorLow[:iTwoLowTrialsGaussDistractorLow,:])


In [53]:
Test = pd.read_csv('../auditory_categorization_longHigh/important_things_not_included_in_assets/allTrials.csv')
Data = pd.read_csv('auditory_categorization_Hc_online_data/auditory_categorization_v3_143976_2022-01-24_22h25.18_d46bc498-64c0-4529-b32c-606dccb68ddf/609553ac4e48356e24a27aed_categorization_task_longHigh_2022-01-21_22h20.32.122.csv');

[trial_tones_expt, trial_behaviour_expt, 
trial_tone_cat_expt, corrans_expt] = dataCleaning(csv_data=Data,
                                                   csv_test=Test,
                                                   total_number_possible_responses=800)
[TrialsGaussHighGaussLow, TrialsGaussHighGaussHigh, 
 TrialsGaussLowGaussHigh, TrialsGaussLowGaussLow, 
 TrialsGaussHighGaussDistractorHigh, TrialsGaussLowGaussDistractorLow,
 TrialsAllHighMinOneSignalHigh, TrialsAllLowMinOneSignalLow,
 TrialsAllHighMinOneSignalLow, TrialsAllLowMinOneSignalHigh,
 TrialsGaussHighDistractorHigh, TrialsGaussLowDistractorLow,
 TrialsGaussHighDistractorLow, TrialsGaussLowDistractorHigh,
 TrialsTwoLowsGaussDistractorLow] = trialCategories(np.log10(trial_tones_expt),
                                                    trial_behaviour_expt, 
                                                    corrans_expt, 
                                                    trial_tone_cat_expt)

print("Accuracy of consecutive simple trials when drawn from opposite gaussian distributions")
print(f"From {len(TrialsGaussHighGaussLow)} trials","High then Low", np.mean(TrialsGaussHighGaussLow[:,3]==TrialsGaussHighGaussLow[:,4])) 
print(f"From {len(TrialsGaussHighGaussHigh)} trials","High then High",np.mean(TrialsGaussHighGaussHigh[:,3]==TrialsGaussHighGaussHigh[:,4]))
print(f"From {len(TrialsGaussLowGaussHigh)} trials","Low then High",np.mean(TrialsGaussLowGaussHigh[:,3]==TrialsGaussLowGaussHigh[:,4]))
print(f"From {len(TrialsGaussLowGaussLow)} trials","Low then Low",np.mean(TrialsGaussLowGaussLow[:,3]==TrialsGaussLowGaussLow[:,4]))
print(f"From {len(TrialsGaussHighGaussDistractorHigh)} trials, expectation of high category based on 1 previous high gaussian trial {np.around(np.mean(TrialsGaussHighGaussDistractorHigh[:,3]==TrialsGaussHighGaussDistractorHigh[:,4]),decimals=2)}")
print(f"From {len(TrialsGaussLowGaussDistractorLow)} trials, expectation of low category based on 1 previous low gaussian trial {np.around(np.mean(TrialsGaussLowGaussDistractorLow[:,3]==TrialsGaussLowGaussDistractorLow[:,4]),decimals=2)}")
print(f"From {len(TrialsGaussLowDistractorLow)} trials, both low but current trial has 1/2 distractor, {np.around(np.mean(TrialsGaussLowDistractorLow[:,3]==TrialsGaussLowDistractorLow[:,4]),decimals=2)}")
print(f"From {len(TrialsGaussHighDistractorHigh)} trials, both high but current trial has 1/2 distractor, {np.around(np.mean(TrialsGaussHighDistractorHigh[:,3]==TrialsGaussHighDistractorHigh[:,4]),decimals=2)}") 
print(f"From {len(TrialsGaussLowDistractorHigh)} trials, low then high and the current trial has 1/2 distractor, {np.around(np.mean(TrialsGaussLowDistractorHigh[:,3]==TrialsGaussLowDistractorHigh[:,4]),decimals=2)}")
print(f"From {len(TrialsGaussHighDistractorLow)} trials, high then low and the current trial has 1/2 distractor, {np.around(np.mean(TrialsGaussHighDistractorLow[:,3]==TrialsGaussHighDistractorLow[:,4]),decimals=2)}")

print("Accuracy of consecutive trials when drawn from opposite categories, all trials are considered")
print(f"From {len(TrialsAllHighMinOneSignalHigh)} trials, high then high, {np.around(np.mean(TrialsAllHighMinOneSignalHigh[:,3]==TrialsAllHighMinOneSignalHigh[:,4]),decimals=2)}")
print(f"From {len(TrialsAllLowMinOneSignalLow)} trials, low then low, {np.around(np.mean(TrialsAllLowMinOneSignalLow[:,3]==TrialsAllLowMinOneSignalLow[:,4]),decimals=2)}")
print(f"From {len(TrialsAllHighMinOneSignalLow)} trials, high then low, {np.around(np.mean(TrialsAllHighMinOneSignalLow[:,3]==TrialsAllHighMinOneSignalLow[:,4]),decimals=2)}")
print(f"From {len(TrialsAllLowMinOneSignalHigh)} trials, low then high, {np.around(np.mean(TrialsAllLowMinOneSignalHigh[:,3]==TrialsAllLowMinOneSignalHigh[:,4]),decimals=2)}") 



Did not respond to:  []
3.0587836612818062 2.358783661281806 2.708783661281806
Accuracy of consecutive simple trials when drawn from opposite gaussian distributions
From 21 trials High then Low 0.7619047619047619
From 47 trials High then High 0.9574468085106383
From 24 trials Low then High 0.9166666666666666
From 10 trials Low then Low 0.9
From 130 trials, expectation of high category based on 1 previous high gaussian trial 0.92
From 19 trials, expectation of low category based on 1 previous low gaussian trial 0.84
From 9 trials, both low but current trial has 1/2 distractor, 0.78
From 83 trials, both high but current trial has 1/2 distractor, 0.89
From 36 trials, low then high and the current trial has 1/2 distractor, 0.78
From 39 trials, high then low and the current trial has 1/2 distractor, 0.64
Accuracy of consecutive trials when drawn from opposite categories, all trials are considered
From 400 trials, high then high, 0.92
From 65 trials, low then low, 0.82
From 159 trials, high 